# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'ht

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [55]:
link_system_prompt = link_system_prompt = """
You are an AI assistant helping to create a brochure for a website.

You are given:
- A website URL
- A list of links found on that website

Your job is to identify the most relevant links for building a brochure. These include pages like 'About', 'Team', 'Careers', 'Products', 'Contact', or any other pages that help describe the organization and what it offers.

For each selected link, return a JSON object with:
- "type": the category of the page (e.g., "about page", "team page", "product page", etc.)
- "url": the full link

Only return the JSON object — do not include any explanation or extra text.

Example 1:
Website: https://example-university.edu  
Links Found:
[
    "https://example-university.edu/",
    "https://example-university.edu/about",
    "https://example-university.edu/contact",
    "https://example-university.edu/blog",
    "https://example-university.edu/wp-login.php",
    "https://example-university.edu/privacy-policy"
]

Output:
{
    "links": [
        {"type": "homepage", "url": "https://example-university.edu/"},
        {"type": "about page", "url": "https://example-university.edu/about"},
        {"type": "contact page", "url": "https://example-university.edu/contact"}
    ]
}

Example 2:
Website: https://tech-startup.io  
Links Found:
[
    "https://tech-startup.io/",
    "https://tech-startup.io/products",
    "https://tech-startup.io/team",
    "https://tech-startup.io/careers",
    "https://tech-startup.io/login",
    "https://tech-startup.io/legal/terms"
]

Output:
{
    "links": [
        {"type": "homepage", "url": "https://tech-startup.io/"},
        {"type": "product page", "url": "https://tech-startup.io/products"},
        {"type": "team page", "url": "https://tech-startup.io/team"},
        {"type": "careers page", "url": "https://tech-startup.io/careers"}
    ]
}

Now do the same for this website:
Website: {{your_website_url}}  
Links Found:
{{your_links_array}}

Output:
"""

In [56]:
print(link_system_prompt)


You are an AI assistant helping to create a brochure for a website.

You are given:
- A website URL
- A list of links found on that website

Your job is to identify the most relevant links for building a brochure. These include pages like 'About', 'Team', 'Careers', 'Products', 'Contact', or any other pages that help describe the organization and what it offers.

For each selected link, return a JSON object with:
- "type": the category of the page (e.g., "about page", "team page", "product page", etc.)
- "url": the full link

Only return the JSON object — do not include any explanation or extra text.

Example 1:
Website: https://example-university.edu  
Links Found:
[
    "https://example-university.edu/",
    "https://example-university.edu/about",
    "https://example-university.edu/contact",
    "https://example-university.edu/blog",
    "https://example-university.edu/wp-login.php",
    "https://example-university.edu/privacy-policy"
]

Output:
{
    "links": [
        {"type": "h

In [57]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [58]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [59]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [60]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/meta-llama/Llama-4-Scout-17B-16E-Instruct',
 '/deepseek-ai/DeepSeek-V3-0324',
 '/Qwen/Qwen2.5-Omni-7B',
 '/all-hands/openhands-lm-32b-v0.1',
 '/openfree/flux-chatgpt-ghibli-lora',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/jamesliu1217/EasyControl_Ghibli',
 '/spaces/VAST-AI/TripoSG',
 '/spaces/Stable-X/Hi3DGen',
 '/spaces/ByteDance/InfiniteYou-FLUX',
 '/spaces',
 '/datasets/open-thoughts/OpenThoughts2-1M',
 '/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset-v1',
 '/datasets/virtuoussy/Multi-subject-RLVR',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets/glaiveai/reasoning-v1-20m',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/g

In [61]:
get_links("https://huggingface.co")

{'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'},
  {'type': 'models page', 'url': 'https://huggingface.co/models'},
  {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'},
  {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'join page', 'url': 'https://huggingface.co/join'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'documentation page', 'url': 'https://huggingface.co/docs'},
  {'type': 'team page', 'url': 'https://huggingface.co/team'},
  {'type': 'contact page', 'url': 'https://huggingface.co/contact'},
  {'type': 'Github page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [62]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [63]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'learn page', 'url': 'https://huggingface.co/learn'}, {'type': 'team page', 'url': 'https://huggingface.co/join'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging F

In [64]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors, and recruits. Respond in markdown.\
# Include details of company culture, customers, and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short, poetic, old-schoolishly toned brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [65]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [66]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'learn page', 'url': 'https://huggingface.co/learn'}, {'type': 'team page', 'url': 'https://huggingface.co/join'}, {'type': 'discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmeta-llama/Llama-4-Scout-17B-16E-Instruct\nUpdated\n1 day ago\n•\n51.1k\n•\n547\ndeepseek-ai/DeepSeek-V3-0324\nUpdated\n12 days ago\n•\n161k\n•\n2.41k\nQwen/Qwen2.5-Omni-7B\nUpdated\n7 days ago\n•\n97.7k\n•\n1.23k\nall-hands/openhands-lm-32b-v0.1\nUpdated\n4 days ago\n•\n4.16k\n•\n294\nopenfree/flux-chatgpt-ghibli-lora\nUpdated\n2 days ago\n•\n7.66k\n•\n298\nBrowse 1M+ models\nSpaces\nRunning\

In [67]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [68]:
create_brochure("HuggingFace", "https://huggingface.com")

Found links: {'links': [{'type': 'homepage', 'url': 'https://huggingface.com/'}, {'type': 'models page', 'url': 'https://huggingface.com/models'}, {'type': 'datasets page', 'url': 'https://huggingface.com/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.com/spaces'}, {'type': 'enterprise page', 'url': 'https://huggingface.com/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.com/pricing'}, {'type': 'join page', 'url': 'https://huggingface.com/join'}, {'type': 'docs page', 'url': 'https://huggingface.com/docs'}, {'type': 'blog page', 'url': 'https://huggingface.com/blog'}, {'type': 'contact page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


```markdown
# Embrace Innovation with Hugging Face

**The AI Community Building the Future**

In realms of code where dreams take flight,  
Hugging Face stands, a beacon of light.  
A hub for minds, both brave and bright,  
Together we harness the power of AI's might.

## Collaboration at Heart

Models and datasets, vast as the sea,  
With over a million, our community agrees.  
From text to images, video, and sound,  
In every creation, new ideas abound.

### For the Creators and Innovators

Join our vibrant **Spaces**, where thoughts intertwine,  
The synergies spark and the visions align.  
Unleash your potential, create without bounds,  
For the future of ML, together, we’ll sound.

## Our Noble Customers

With enterprises grand and visions sublime,  
From Google to Amazon, partners in crime.  
Over fifty thousand organizations stride,  
In the world of AI, we stand side by side.

### Suitable for All

For startups and giants alike, we have the tools,  
A supportive embrace, where everyone rules.  
With enterprise features and security profound,  
Together, we build a future unbound.

## Careers Worth Pursuing

In our halls, your passion will soar,  
Join hands with pioneers and explore.  
A culture of kindness, innovation, and grace,  
At Hugging Face, you’ll find your place.

### Call to Adventure

So come forth, dear innovators,  
Unite with the curious creators,  
At Hugging Face, we craft and inspire,  
Together, we’ll spark the future — aspire!

**Join Us Today and Shape Tomorrow!**  
- Explore our Models  
- Create in Spaces  
- Engage with Datasets  
- Foster your Career  
- Become part of our legacy

*Hugging Face: Where Future Meets Community.*
```


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [69]:
def stream_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    return response.choices[0].message.content

In [70]:
sytem_prompt_persian = "You are an assistant that reads a brochure of a website in English \
                        and translate it into Persian language. Respond in markdown. \
                         Include all the info in the original brochure and try to replicate the writing style in it."

In [71]:
def stream_brochure_persian(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt_persian},
            {"role": "user", "content": stream_brochure(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        response = response.replace ("markdown", "").replace("```", "")
        # yield response
        update_display(Markdown(response), display_id=display_handle.display_id)
        # yield chunk

In [72]:
stream_brochure_persian("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'team page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'join page', 'url': 'https://huggingface.co/join'}, {'type': 'learn page', 'url': 'https://huggingface.co/learn'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# **Hugging Face: آینده‌ی هوش مصنوعی در انتظار است!**

در وسعت داده‌ها، جایی که رویاها پرواز می‌کنند،  
جامعه‌ای پرجنب و جوش، درخشنده و روشن،  
Hugging Face، چراغی روشن، راه را هدایت می‌کند،  
در دنیای وسیع روزهای نوین هوش مصنوعی.  

## **پناهگاهی برای نوآوران**  
با مدل‌هایی فراوان، بیش از یک میلیون برای جستجو،  
در داده‌های انبوه، جمعیتی به پا می‌خیزد،  
از متن تا تصویر، از صدا تا کد،  
دارایی‌ای بافته شده، در حالی‌که دانش تقدیم می‌شود.  

با **Spaces** آماده برای همکاری‌های بزرگ،  
جایی که ذهن‌ها گرد هم می‌آیند، دست به دست هم می‌دهند،  
ایجاد کن، آفرینش کن، بگذار ایده‌هایت آشکار شوند،  
در این پناهگاه پرورش‌دهنده، جادو را ببین!  

## **برای مشتریان و مشاغل**  
در خط مقدم، نام‌هایی بزرگ و شگرف،  
Meta، Amazon، و Google، گنجینه‌ای برای دیدن،  
نیروهای خود را با غول‌ها همراه کن، جایی که بینش شعله‌ور است،  
در همکاری‌هایی پرشور با بینش‌های نوآورانه.  

اگر در جستجوی شغلی هستی، جایی که اشتیاق با هنر ملاقات کند،  
Hugging Face دعوت می‌کند، سفر تو به اوج خواهد رسید،  
برای آنان که جرات رویا داشتن دارند و می‌خواهند بیافرینند،  
فرهنگ باز انتظار تو را می‌کشد دروازه‌ها را گشوده.  

## **میراث اوپن‌سورس ما**  
تعهد به اشتراک‌گذاری، هر پروژه‌ای هدیه‌ای،  
Transformers، Diffusers، روح ما را صیقل می‌دهند،  
از ابزارهای پیشرفته تا کتابخانه‌های بزرگ،  
در شکل‌دهی به آینده‌ای که برنامه‌ریزی کرده‌ایم، با ما همراه شو!  

با **سؤالات متداول** و **مستندات** برای هدایت راهت،  
کاوش کن در **وبلاگ** ما جایی که بینش‌ها منتقل می‌شوند،  
در سفری با هم، دست در دست می‌دویم،  
شگفتی‌ها را در Hugging Face می‌سازیم!  

## **با ما ارتباط برقرار کن**  
پس پیشقدم شو، دوست عزیز، پرش را به انجام برسان،  
در قلب هوش مصنوعی، آنچه را که می‌کاری، درو کن،  
جزئی از ما شو، درهای امروز به رویت گشوده‌اند،  
در Hugging Face، بگذار آرزوهایت پرواز کنند!  

**وب‌سایت:** [Hugging Face](https://huggingface.co)  
**به جنبش بپیوندید:** ثبت‌نام کنید و بزرگ بیندیشید!  


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>

In [29]:
import gradio as gr # oh yeah!

In [54]:
# Inputs and Outputs

view = gr.Interface(
    fn=stream_brochure,
    inputs=[gr.Textbox(label="Which website do you want to see a brochure of?:", lines=2), gr.Textbox(label="Give the link of the website:", lines=2)],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7875

To create a public link, set `share=True` in `launch()`.


Found links: {'links': [{'type': 'about page', 'url': 'https://twosetviolin.com/about'}, {'type': 'team page', 'url': 'https://twosetviolin.com/team'}, {'type': 'contact page', 'url': 'https://twosetviolin.com/contact'}]}
